### Title

## Project description
# words

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
usage = pd.read_csv("../data/raw/usage_reports.csv")
rates = pd.read_csv("../data/raw/royalty_rates.csv")
payments = pd.read_csv("../data/raw/payments.csv")
artists = pd.read_csv("../data/raw/artist_lookup.csv")
usage.head()

,usage_report_id,report_month,platform,territory,track_id,isrc,track_title,artist_id,artist_name,release_date,stream_type,streams,unique_listeners,avg_stream_seconds,reported_currency
0,UR20000001,2023-07,Spotify,FR,10362,ISRC4538002,No Signal,AR0016,Indigo Santos,2019-08-22,free,950,333,232.322142,EUR
1,UR20000002,2023-10,Amazon Music,GB,10536,ISRC1936197,Neon Dreams,AR0025,Kai Atlas,2020-06-27,free,656,124,174.848842,GBP
2,UR20000003,2023-11,Apple Music,GB,11095,ISRC3578005,Neon Dreams,AR0052,Nico Koi,2017-06-18,free,1698,418,184.835507,GBP
3,UR20000004,2023-09,YouTube Music,CA,10478,ISRC4030872,Afterglow,AR0021,Iris Stone & The Collective,2021-06-20,premium,1442,270,206.766053,CAD
4,UR20000005,2025-12,Apple Music,CA,11135,ISRC1204488,City Lights (Edit),AR0055,Nico Ryder Jr.,2024-03-15,premium,4760,1581,154.728930,CAD


In [5]:
usage['artist_name'].value_counts()

artist_name
Theo Vega       2815
Nico Stone      2779
Sage Atlas      2763
Indigo Blaze    2761
Kai Blaze       2753
                ... 
Mila Reign       652
Jade Atlas       649
Kai Sol          648
Theo Haze        644
Mila Vega        640
Name: count, Length: 100, dtype: int64